### Generate Training Data
Following the tutorial article: http://monik.in/a-noobs-guide-to-implementing-rnn-lstm-using-tensorflow/

In [1]:
import numpy as np
from random import shuffle

In [27]:
train_input = ['{0:020b}'.format(i) for i in range(2**20)]
shuffle(train_input)

In [28]:
#train_input[0]
# '01110010100000001101'

In [29]:
train_input = [map(int,i) for i in train_input]

In [30]:
#train_input[0]
# <map at 0x112bef668>

In [31]:
ti = []
for i in train_input:
    temp_list = []
    for j in i:
        temp_list.append([j])
    ti.append(np.array(temp_list))
train_input = ti

In [7]:
#train_input[0]
# array([[0],[0],[1],[0],[0],[1],[0],[1],[1],[0],[0],[0],[1],[1],[1],[1],[1],[1],[0],[0]])

In [32]:
#train_output = []
train_output_R = []
for i in train_input:
    count = 0
    for j in i:
        if j[0]==1:
            count+=1
    train_output_R.append([count])
    #temp_list = [0]*21
    #temp_list[count] = 1
    #train_output.append(temp_list)

In [38]:
#test_output[0]
# [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
train_input_R = train_input # backup
len(train_input),len(train_output_R)
#train_input[0]
#train_output_R[0]

(1048576, 1048576)

In [39]:
SPLIT = 0.01        # let's see how many are actually needed!
NUM_EXAMPLES = int(SPLIT*len(train_input))  # 10485
test_input = train_input_R[NUM_EXAMPLES:]
test_output = train_output_R[NUM_EXAMPLES:]
train_input = train_input_R[:NUM_EXAMPLES]
train_output = train_output_R[:NUM_EXAMPLES]

## Tensorflow starts

In [34]:
import tensorflow as tf
from tensorflow.contrib import rnn

In [32]:
data = tf.placeholder(tf.float32,[None,20,1])
target = tf.placeholder(tf.float32,[None,21])

In [36]:
num_hidden = 24 # Why?
cell = rnn.LSTMCell(num_hidden, state_is_tuple = True) # rnn has been moved to tensoflow.contrib

In [37]:
# define the executions for later.
val,state = tf.nn.dynamic_rnn(cell,data,dtype=tf.float32)
# transpose
val = tf.transpose(val,[1,0,2])
# now take only last output (after 20 unfoldings)
last = tf.gather(val,int(val.get_shape()[0])-1)

In [39]:
weight = tf.Variable(tf.truncated_normal([num_hidden,int(target.get_shape()[1])]))
bias = tf.Variable(tf.constant(0.1,shape=[int(target.get_shape()[1])]))
prediction = tf.nn.softmax(tf.matmul(last,weight)+bias)

In [40]:
cross_entropy = - tf.reduce_sum(target*tf.log(tf.clip_by_value(prediction,1e-10,1.0)))
optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(cross_entropy)

/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [41]:
mistakes = tf.not_equal(tf.argmax(target,1),tf.argmax(prediction,1))
error = tf.reduce_mean(tf.cast(mistakes,tf.float32))

## Running

In [43]:
init_op = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init_op)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [ ]:
batch_size = 1000
n_batches = int(len(train_input)/batch_size)
epoch = 500
for i in range(epoch):
    ptr = 0
    for j in range(n_batches):
        inp, out = train_input[ptr:ptr+batch_size],train_output[ptr:ptr+batch_size]
        ptr+=batch_size
        sess.run(minimize,{data:inp,target:out})
    print("Epoch - ",str(i+1))
    incorrect = sess.run(error,{data: test_input, target: test_output})
    print('Epoch {:2d} error {:3.1f}%'.format(i+1,100*incorrect))
    print(sess.run(prediction,{data: [[[1],[0],[0],[1],[1],[0],[1],[1],[1],[0],[1],[0],[0],[1],[1],[0],[1],[1],[1],[0]]]}))
sess.close()

Epoch -  1
Epoch  1 error 83.5%
[[ 0.03025684  0.04333294  0.02665082  0.04626855  0.05087003  0.02715817
   0.03947088  0.04623447  0.05392186  0.07287259  0.05093984  0.09283719
   0.06219664  0.07470414  0.04053288  0.04036006  0.05289216  0.02263099
   0.05101996  0.04492676  0.02992232]]
Epoch -  2
Epoch  2 error 82.6%
[[ 0.02342945  0.02327973  0.01815589  0.03191546  0.05073449  0.01809984
   0.03027917  0.04485922  0.0643153   0.1076844   0.05700868  0.11431289
   0.0878066   0.08183929  0.03333314  0.03962979  0.03797347  0.01480325
   0.04517341  0.04798603  0.02738049]]
Epoch -  3
Epoch  3 error 83.4%
[[ 0.01178274  0.00779412  0.00748707  0.01400196  0.03795063  0.0077305
   0.02015468  0.03859974  0.07720038  0.17424771  0.08648838  0.15873228
   0.09780644  0.08801519  0.02447713  0.02403957  0.0210625   0.00457062
   0.03161613  0.04894936  0.01729286]]
Epoch -  4
Epoch  4 error 84.1%
[[ 0.0051799   0.00271647  0.00281337  0.00396086  0.02201746  0.00369792
   0.01561055

Epoch 24 error 82.4%
[[  4.11853747e-04   5.35666069e-04   2.95324542e-04   5.78077044e-04
    3.66566493e-03   1.53242685e-02   3.66292000e-02   7.16616139e-02
    1.12383783e-01   1.60526678e-01   1.76453009e-01   1.65158570e-01
    1.18327707e-01   7.90448040e-02   3.72070484e-02   1.18040349e-02
    4.23857290e-03   9.70168549e-05   2.03722343e-03   2.27250275e-03
    1.34742551e-03]]
Epoch -  25
Epoch 25 error 82.4%
[[  3.91855050e-04   5.15148568e-04   2.85881863e-04   5.89768868e-04
    3.65378777e-03   1.53728165e-02   3.67090367e-02   7.16685951e-02
    1.12726443e-01   1.60401717e-01   1.76682308e-01   1.64706752e-01
    1.18591085e-01   7.89490715e-02   3.71830389e-02   1.19221713e-02
    4.22793394e-03   9.73662682e-05   1.91662391e-03   2.14793603e-03
    1.26067491e-03]]
Epoch -  26
Epoch 26 error 82.4%
[[  3.73632065e-04   4.95764369e-04   2.77221727e-04   6.02678745e-04
    3.64563474e-03   1.54137621e-02   3.67795154e-02   7.16645345e-02
    1.13049939e-01   1.60286859

Epoch -  45
Epoch 45 error 82.4%
[[  1.90877559e-04   2.59373977e-04   1.84930628e-04   9.31210991e-04
    3.69660463e-03   1.56571791e-02   3.72594781e-02   7.10034892e-02
    1.16761118e-01   1.59483060e-01   1.79657042e-01   1.59089997e-01
    1.22379586e-01   7.73077831e-02   3.64270061e-02   1.30271269e-02
    4.24246443e-03   1.61480362e-04   8.05595191e-04   9.81098041e-04
    4.93491767e-04]]
Epoch -  46
Epoch 46 error 82.4%
[[  1.85643061e-04   2.51683145e-04   1.82046337e-04   9.46371991e-04
    3.70102935e-03   1.56601183e-02   3.72608453e-02   7.09634870e-02
    1.16872326e-01   1.59491703e-01   1.79721102e-01   1.58935130e-01
    1.22521013e-01   7.72495344e-02   3.63856852e-02   1.30521078e-02
    4.24385071e-03   1.67018690e-04   7.79633061e-04   9.52837581e-04
    4.76809248e-04]]
Epoch -  47
Epoch 47 error 82.4%
[[  1.80642528e-04   2.44319497e-04   1.79271985e-04   9.60853067e-04
    3.70535091e-03   1.56627595e-02   3.72610241e-02   7.09236562e-02
    1.16978034e-01 

Epoch -  66
Epoch 66 error 82.5%
[[  1.15741372e-04   1.49224274e-04   1.41366269e-04   1.10837980e-03
    3.76552413e-03   1.56741440e-02   3.71325202e-02   7.02206716e-02
    1.18264109e-01   1.60075128e-01   1.80219144e-01   1.57032236e-01
    1.24801658e-01   7.64525756e-02   3.56073603e-02   1.33628566e-02
    4.27180156e-03   3.04922927e-04   4.49167273e-04   5.80413092e-04
    2.71071040e-04]]
Epoch -  67
Epoch 67 error 82.5%
[[  1.13408336e-04   1.45886777e-04   1.39934607e-04   1.11099763e-03
    3.76767386e-03   1.56731978e-02   3.71219926e-02   7.01869428e-02
    1.18305154e-01   1.60114646e-01   1.80222169e-01   1.56976625e-01
    1.24893025e-01   7.64270276e-02   3.55720073e-02   1.33723747e-02
    4.27325629e-03   3.12137767e-04   4.38720279e-04   5.68067422e-04
    2.64715432e-04]]
Epoch -  68
Epoch 68 error 82.5%
[[  1.11149930e-04   1.42665449e-04   1.38544710e-04   1.11330545e-03
    3.76974582e-03   1.56721380e-02   3.71113271e-02   7.01534525e-02
    1.18344247e-01 

Epoch -  87
Epoch 87 error 82.6%
[[  7.89552651e-05   9.78708631e-05   1.18545162e-04   1.12494733e-03
    3.79755907e-03   1.56331453e-02   3.68931480e-02   6.95803165e-02
    1.18832402e-01   1.60916820e-01   1.80166483e-01   1.56237632e-01
    1.26405522e-01   7.60959685e-02   3.49472426e-02   1.35012111e-02
    4.29807650e-03   4.24187834e-04   2.89635878e-04   3.86127620e-04
    1.74254848e-04]]
Epoch -  88
Epoch 88 error 82.6%
[[  7.76884699e-05   9.61542537e-05   1.17769312e-04   1.12478447e-03
    3.79855419e-03   1.56304128e-02   3.68814915e-02   6.95531741e-02
    1.18848316e-01   1.60955429e-01   1.80161029e-01   1.56213716e-01
    1.26468107e-01   7.60864466e-02   3.49199213e-02   1.35053461e-02
    4.29901527e-03   4.27813211e-04   2.84346926e-04   3.79472360e-04
    1.71048596e-04]]
Epoch -  89
Epoch 89 error 82.6%
[[  7.64539946e-05   9.44849089e-05   1.17015938e-04   1.12459646e-03
    3.79950856e-03   1.56276245e-02   3.68698612e-02   6.95263520e-02
    1.18863247e-01 

...

Epoch -  339
Epoch 339 error 82.8%
[[  9.26704433e-06   9.16085355e-06   1.03266553e-04   1.00835250e-03
    3.80273955e-03   1.57569200e-02   3.53416577e-02   6.23566322e-02
    1.18991531e-01   1.67751193e-01   1.78079620e-01   1.58255413e-01
    1.30190387e-01   7.72218034e-02   3.18080001e-02   1.45388925e-02
    4.22217278e-03   4.71083855e-04   2.56805379e-05   4.03495251e-05
    1.58683306e-05]]
Epoch -  340
Epoch 340 error 82.8%
[[  9.21529045e-06   9.10367362e-06   1.03191647e-04   1.00743456e-03
    3.80345457e-03   1.57571658e-02   3.53355631e-02   6.23161793e-02
    1.19001731e-01   1.67801484e-01   1.78069532e-01   1.58272624e-01
    1.30165905e-01   7.72173554e-02   3.18044871e-02   1.45545453e-02
    4.21880046e-03   4.70816449e-04   2.55207851e-05   4.01071193e-05
    1.57616778e-05]]
Epoch -  341
Epoch 341 error 82.7%
[[  9.16385761e-06   9.04699482e-06   1.03116305e-04   1.00652105e-03
    3.80420522e-03   1.57572478e-02   3.53292413e-02   6.22764304e-02
    1.1901170

Epoch -  360
Epoch 360 error 82.8%
[[  8.25016195e-06   8.06629032e-06   1.01882557e-04   9.90429195e-04
    3.82517162e-03   1.57324374e-02   3.51605453e-02   6.16549328e-02
    1.19151242e-01   1.68695197e-01   1.78008348e-01   1.58436060e-01
    1.29749984e-01   7.72165582e-02   3.17153744e-02   1.48617048e-02
    4.14573913e-03   4.66149708e-04   2.26036427e-05   3.55974735e-05
    1.37592215e-05]]
Epoch -  361
Epoch 361 error 82.8%
[[  8.20537753e-06   8.01958959e-06   1.01836813e-04   9.89650376e-04
    3.82658863e-03   1.57302581e-02   3.51492092e-02   6.16290011e-02
    1.19155906e-01   1.68733388e-01   1.78008109e-01   1.58437461e-01
    1.29735380e-01   7.72220120e-02   3.17088515e-02   1.48767084e-02
    4.14192071e-03   4.65952180e-04   2.24715786e-05   3.53897958e-05
    1.36654926e-05]]
Epoch -  362
Epoch 362 error 82.8%
[[  8.16088323e-06   7.97329631e-06   1.01793026e-04   9.88874002e-04
    3.82802496e-03   1.57280639e-02   3.51376161e-02   6.16036877e-02
    1.1916034

Epoch -  381
Epoch 381 error 82.9%
[[  7.35722051e-06   7.14288217e-06   1.01026861e-04   9.74238967e-04
    3.85836628e-03   1.56862289e-02   3.48779820e-02   6.12438060e-02
    1.19162753e-01   1.69410214e-01   1.77946061e-01   1.58354938e-01
    1.29645720e-01   7.74319470e-02   3.15279476e-02   1.51751451e-02
    4.06403840e-03   4.61701071e-04   2.00066861e-05   3.15277211e-05
    1.18877924e-05]]
Epoch -  382
Epoch 382 error 82.9%
[[  7.31690943e-06   7.10079939e-06   1.00977348e-04   9.73492221e-04
    3.86015046e-03   1.56839509e-02   3.48622724e-02   6.12314083e-02
    1.19157955e-01   1.69441000e-01   1.77940115e-01   1.58345267e-01
    1.29650846e-01   7.74461702e-02   3.15171182e-02   1.51902540e-02
    4.06008773e-03   4.61450574e-04   1.98888720e-05   3.13486380e-05
    1.18035059e-05]]
Epoch -  383
Epoch 383 error 82.9%
[[  7.27682709e-06   7.05887805e-06   1.00926576e-04   9.72752168e-04
    3.86195956e-03   1.56816449e-02   3.48463692e-02   6.12197518e-02
    1.1915278

Epoch -  402
Epoch 402 error 82.9%
[[  6.57469673e-06   6.31234707e-06   9.99034601e-05   9.60213947e-04
    3.90107208e-03   1.56253632e-02   3.45135033e-02   6.11215383e-02
    1.18980125e-01   1.69982135e-01   1.77787408e-01   1.58118427e-01
    1.29842356e-01   7.77712390e-02   3.12876664e-02   1.55019257e-02
    3.98202799e-03   4.56226466e-04   1.76306621e-05   2.81184239e-05
    1.02393469e-05]]
Epoch -  403
Epoch 403 error 82.9%
[[  6.54132828e-06   6.27647159e-06   9.98550313e-05   9.59614874e-04
    3.90330609e-03   1.56215718e-02   3.44947651e-02   6.11227527e-02
    1.18967839e-01   1.70004234e-01   1.77778348e-01   1.58107951e-01
    1.29854321e-01   7.77890980e-02   3.12756598e-02   1.55179631e-02
    3.97821143e-03   4.55968810e-04   1.75238911e-05   2.79756587e-05
    1.01677624e-05]]
Epoch -  404
Epoch 404 error 82.9%
[[  6.50833999e-06   6.24100858e-06   9.98077739e-05   9.59017081e-04
    3.90554895e-03   1.56176789e-02   3.44759338e-02   6.11245185e-02
    1.1895521

Epoch -  423
Epoch 423 error 82.9%
[[  5.95203255e-06   5.64440961e-06   9.91849447e-05   9.47697379e-04
    3.94883938e-03   1.55253112e-02   3.41063552e-02   6.12717569e-02
    1.18667245e-01   1.70317814e-01   1.77578002e-01   1.57957867e-01
    1.30123869e-01   7.81686977e-02   3.10242455e-02   1.58464704e-02
    3.90433776e-03   4.50823864e-04   1.55417456e-05   2.54843890e-05
    8.86958514e-06]]
Epoch -  424
Epoch 424 error 82.9%
[[  5.92627021e-06   5.61699380e-06   9.91680645e-05   9.47077991e-04
    3.95111367e-03   1.55194532e-02   3.40863839e-02   6.12852536e-02
    1.18649788e-01   1.70326754e-01   1.77567080e-01   1.57953382e-01
    1.30139142e-01   7.81885684e-02   3.10111381e-02   1.58632807e-02
    3.90078616e-03   4.50559077e-04   1.54506452e-05   2.53769631e-05
    8.81107189e-06]]
Epoch -  425


## Attempt as Regression

In [11]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [185]:
SPLIT = 0.001        # let's see how many are actually needed!
NUM_EXAMPLES = int(SPLIT*len(train_input_R))  # 1048
test_input = train_input_R[NUM_EXAMPLES:]
test_output = train_output_R[NUM_EXAMPLES:]
train_input = train_input_R[:NUM_EXAMPLES]
train_output = train_output_R[:NUM_EXAMPLES]

In [186]:
train_input_1 = np.array(train_input)
train_input_1.shape
test_input_1 = np.array(test_input)
#test_input_1.shape
train_output_1 = np.array(train_output)
#train_output_1.shape
test_output_1 = np.array(test_output)
#test_output_1.shape
train_input_1.shape

(1048, 20, 1)

In [187]:
model2 = keras.Sequential()
model2.add(keras.layers.SimpleRNN(1, activation=tf.nn.relu,use_bias=True))
#model2.add(keras.layers.Dense(1, activation=tf.nn.relu,use_bias=False))

In [188]:
optimizer = tf.train.RMSPropOptimizer(0.001)

model2.compile(loss='mse',
            optimizer=optimizer,
            metrics=['mae'])

#model1.summary()

In [192]:
EPOCHS = 100
history = model2.fit(train_input_1, train_output_1, epochs=EPOCHS)

Epoch 1/100
1048/1048 [==============================] - 0s 147us/step - loss: 0.9681 - mean_absolute_error: 0.7796
Epoch 2/100
1048/1048 [==============================] - 0s 163us/step - loss: 0.9027 - mean_absolute_error: 0.7587
Epoch 3/100
1048/1048 [==============================] - 0s 143us/step - loss: 0.8527 - mean_absolute_error: 0.7361
Epoch 4/100
1048/1048 [==============================] - 0s 135us/step - loss: 0.7979 - mean_absolute_error: 0.7092
Epoch 5/100
1048/1048 [==============================] - 0s 134us/step - loss: 0.7358 - mean_absolute_error: 0.6837
Epoch 6/100
1048/1048 [==============================] - 0s 133us/step - loss: 0.6770 - mean_absolute_error: 0.6562
Epoch 7/100
1048/1048 [==============================] - 0s 135us/step - loss: 0.6235 - mean_absolute_error: 0.6255
Epoch 8/100
1048/1048 [==============================] - 0s 152us/step - loss: 0.5706 - mean_absolute_error: 0.6023
Epoch 9/100
1048/1048 [==============================] - 0s 135us/step -

In [193]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_11 (SimpleRNN)    (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [169]:
i = 1
test_input_1[i] # count is 8.
x = model2.predict(np.array([test_input_1[i]]))
round(list(x[0])[0]),test_output_1[i]

(9.0, array([9]))

In [170]:
[loss, mae] = model2.evaluate(test_input_1, test_output_1, verbose=0)

print("Testing set Mean Abs Error: ${:7.2f}".format(mae))

Testing set Mean Abs Error: $   0.09


In [171]:
len(test_input_1)

1038091

In [196]:
# Self-designed test (must be a predefined function I'm sure)
# Count number of wrongs. Not MAE. Round off model prediction. If == answer, YES, else NO.

# Have once run this predictions. No need to, now. For this model.
%time predictions = model2.predict(np.array(test_input_1))
score_int,score_round = 0,0
for i,guess in enumerate(predictions):
    guess_int = int(guess)
    guess_round = int(round(list(guess)[0]))
    label = test_output_1[i]
    if guess_int != label:
        score_int+=1
    if guess_round != label:
        score_round+=1
        #print(guess,label)
#    if i!=0 and i%100000==0:
#        #print(i,'\t {:7.2f} \t {}'.format(float(score_int*100)/i,float(score_round*100)/i))
#        print(i,'\t',score_int,'\t',score_round)
print(i,'\t',score_int,'\t',score_round)

CPU times: user 1min 24s, sys: 31 s, total: 1min 55s
Wall time: 48 s
1047527 	 1047231 	 0


In [195]:
model2.save('model_6.h5')

In [174]:
i = 20
x = [0]*20
list(model2.predict(np.array([test_input_1[i]]))[0])[0],test_output_1[i][0]

(12.889934, 13)

In [194]:
type(model2)
#model2.weights
model2.get_weights()

[array([[1.0024158]], dtype=float32),
 array([[0.9996718]], dtype=float32),
 array([-0.00070112], dtype=float32)]